# Classification:

While Linear Regression is used to predict a numeric value, classification is used to separate data into different labels. 

In this example, I will use the TensorFlow estimator to classify flowers.

Note that there won't be as many notes, as most of it is covered in previous lessons (and are therefore in the previous Jupyter Notebook)

# Imports and Setup:

In [1]:
#importing various libraries into this session:

from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pandas as pd



# Data Set:
This specific dataset will seperate flowers into 3 different classes/species:
* Setosa
* Versicolor
* Virginica

Information is given for each flower, this includes:
* Sepal Length
* Sepal Width
* Petal Length
* Petal Width

In [ ]:
#Creating some constants:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']